# Базы данных: итоговый проект
## Выполнила Елизавета Клыкова (БКЛ-181)

Критерии проекта:
1. ✔ Серверная SQL-СУБД
2. ✔ Никак не меньше 3 таблиц
3. ✔ Красивая структура БД, разумно хранящая информацию выбранной предметной области, и общая красота решения (2 балла)
4. ✔ Какой-нибудь интерфейс (например, в Юпитеровской тетрадке) для ввода, модификации, удаления и выборки данных из базы (1 балл)
5. ✔ SELECT с несколькими (от трех) таблицами в условии выборки (1 балл)
6. ✔ Вложенные SELECT (2 балла)
7. ✔ Запросы с JOIN (2 балла)
8. ✔ Агрегация, сортировки (2 балл)

#### Импорт библиотек

In [1]:
# !pip install mysql-connector-python

In [2]:
import re
import time
import random
import requests
import pandas as pd
import mysql.connector

from bs4 import BeautifulSoup
from tqdm.auto import tqdm

session = requests.session()

## Сбор данных
В качестве источника выбран ресурс [Archive of Our Own](https://archiveofourown.org/) -- мультиязычный сайт любительской прозы и поэзии.

In [3]:
def get_links_to_works(page_link):
    page = session.get(page_link).text
    soup = BeautifulSoup(page, 'html.parser')
    blocks = soup.find_all('div', {'class': 'header module'})
    links = []
    for block in tqdm(blocks):
        link = block.find('h4', {'class': 'heading'}).find('a').attrs['href']
        full_link = 'https://archiveofourown.org' + \
            link + '?view_full_work=true'
        links.append(full_link)
    time.sleep(random.randint(3, 5))
    return links

In [4]:
links_to_works = []
for i in range(3):
    url = 'https://archiveofourown.org/works/search?commit=Search&page={}&utf8=%E2%9C%93&work_search%5Bbookmarks_count%5D=&work_search%5Bcharacter_names%5D=&work_search%5Bcomments_count%5D=&work_search%5Bcomplete%5D=&work_search%5Bcreators%5D=&work_search%5Bcrossover%5D=&work_search%5Bfandom_names%5D=&work_search%5Bfreeform_names%5D=&work_search%5Bhits%5D=&work_search%5Bkudos_count%5D=&work_search%5Blanguage_id%5D=&work_search%5Bquery%5D=&work_search%5Brating_ids%5D=&work_search%5Brelationship_names%5D=&work_search%5Brevised_at%5D=&work_search%5Bsingle_chapter%5D=0&work_search%5Bsort_column%5D=kudos_count&work_search%5Bsort_direction%5D=desc&work_search%5Btitle%5D=&work_search%5Bword_count%5D=.html'.format(i+1)
    links_to_works.extend(get_links_to_works(url))

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [5]:
def get_fanfic_info(fanfic_link):
    page = session.get(fanfic_link).text
    soup = BeautifulSoup(page, 'html.parser')

    id_fanfic = int(fanfic_link.split('/')[-1].split('?')[0])

    # header info
    fanfic_header = soup.find('div', {'id': 'workskin'})
    title = fanfic_header.find('h2', {'class': 'title heading'}).text.strip()

    author_link = 'https://archiveofourown.org' + fanfic_header.find(
        'a', {'rel': 'author'}).attrs['href']
    author_link = re.sub('/pseuds/.*', '/profile', author_link)

    summary = fanfic_header.find('div', {'class': 'summary module'})
    if summary:
        summary = summary.find('blockquote').text.strip()
    else:
        summary = ''

    # meta info
    fanfic_meta = soup.find('dl', {'class': 'work meta group'})
    rating = [rt.text for rt in fanfic_meta.find(
        'dd', {'class': 'rating tags'}).find_all('a')][0]
    try:
        category = [ct.text for ct in fanfic_meta.find(
            'dd', {'class': 'category tags'}).find_all('a')][0]
        if 'None' in category:
            category = None
    except AttributeError:
        category = None
    warnings = [wg.text for wg in fanfic_meta.find(
        'dd', {'class': 'warning tags'}).find_all('a')]
    fandoms = [fd.text for fd in fanfic_meta.find(
        'dd', {'class': 'fandom tags'}).find_all('a')]
    try:
        tags = [tg.text for tg in fanfic_meta.find(
            'dd', {'class': 'freeform tags'}).find_all('a')]
    except AttributeError:
        tags = None
    try:
        chars = [ch.text for ch in fanfic_meta.find(
            'dd', {'class': 'character tags'}).find_all('a')]
    except AttributeError:
        chars = None
    try:
        pairings = [pr.text for pr in fanfic_meta.find(
            'dd', {'class': 'relationship tags'}).find_all('a')]
    except AttributeError:
        pairings = None
    language = fanfic_meta.find('dd', {'class': 'language'}).text.strip()

    # stats
    fanfic_stats = soup.find('dl', {'class': 'stats'})
    pub_date = fanfic_stats.find('dd', {'class': 'published'}).text
    try:
        upd_date = fanfic_stats.find('dd', {'class': 'status'}).text
    except AttributeError:
        upd_date = "0000-00-00"
    words = int(fanfic_stats.find('dd', {'class': 'words'}).text)
    chapts = fanfic_stats.find('dd', {'class': 'chapters'}).text.split('/')
    chapt_num = int(chapts[0])
    if '?' not in chapts[-1]:
        status = 1
    else:
        status = 0
    try:
        kudos = int(fanfic_stats.find('dd', {'class': 'kudos'}).text)
    except AttributeError:
        kudos = 0
    hits = int(fanfic_stats.find('dd', {'class': 'hits'}).text)

    # chapters
    chapter_list = []
    content = soup.find('div', {'id': 'chapters', 'role': 'article'})
    multi_chapter = content.find('div', {'class': 'chapter'})
    if not multi_chapter:
        chapter_text = re.sub('^Work Text:', '', content.text.strip())
        chapter_info = {'id_fanfic': id_fanfic,
                        'chapter_num': 0,
                        'chapter_name': None,
                        'chapter_link': None,
                        'chapter_summary': None,
                        'chapter_text': chapter_text.strip()}
        chapter_list.append(chapter_info)
    else:
        chapters = content.find_all('div', {'class': 'chapter'})
        chapter_num = 1
        for chapter in chapters:
            chapter_meta = chapter.find(
                'div', {'class': 'chapter preface group'})
            chapter_content = chapter.find(
                'div', {'class': 'userstuff module', 'role': 'article'})
            if chapter_meta and chapter_content:
                chapter_name = chapter_meta.find('h3', {'class': 'title'}).text
                chapter_link = 'https://archiveofourown.org' + \
                    chapter_meta.find(
                        'h3', {'class': 'title'}).find('a').attrs['href']
                try:
                    chapter_sum = chapter_meta.find(
                        'div', {'id': 'summary', 'class': 'summary module'}
                    ).text
                    chapter_sum = re.sub('^Summary:', '',
                                         chapter_sum).strip()
                except AttributeError:
                    chapter_sum = ''
                chapter_text = chapter_content.text
                chapter_text = re.sub('^Chapter Text', '',
                                      chapter_text.strip())
                chapter_info = {'id_fanfic': id_fanfic,
                                'chapter_num': chapter_num,
                                'chapter_name': chapter_name.strip(),
                                'chapter_link': chapter_link,
                                'chapter_summary': chapter_sum,
                                'chapter_text': chapter_text.strip()}
                chapter_list.append(chapter_info)
                chapter_num += 1

    # create a dictionary
    fanfic_dict = {'id_fanfic': id_fanfic,
                   'fanfic_link': fanfic_link,
                   'title': title,
                   'author_link': author_link,
                   'summary': summary,
                   'rating': rating,
                   'category': category,
                   'warnings': warnings,
                   'tags': tags,
                   'fandoms': fandoms,
                   'characters': chars,
                   'pairings': pairings,
                   'language': language,
                   'pub_date': pub_date,
                   'upd_date': upd_date,
                   'words': words,
                   'chapt_num': chapt_num,
                   'status': status,
                   'kudos': kudos,
                   'hits': hits,
                   'chapters': chapter_list
                   }

    return id_fanfic, fanfic_dict

In [6]:
def get_author_info(author_link):
    page = session.get(author_link).text
    soup = BeautifulSoup(page, 'html.parser')

    # name
    author_name = soup.find('div',
                            {'class': 'primary header module'}
                            ).find('h2').text.strip()

    # meta: works, series, gifts
    author_meta = soup.find('div', {'id': 'dashboard'})
    meta = []
    for lst in author_meta.find_all('ul', {'class': 'navigation actions'}):
        meta.extend([el.text for el in lst.find_all('li')])
    meta = '; '.join(meta)
    works = int(re.search(r'Works \((.*?)\)', meta).group(1))
    series = int(re.search(r'Series \((.*?)\)', meta).group(1))
    gifts = int(re.search(r'Gifts \((.*?)\)', meta).group(1))

    # meta: bio
    try:
        bio = soup.find('div', {'class': 'bio module'}).text
        bio = re.sub('^Bio', '', bio.strip()).strip()
    except AttributeError:
        bio = ''

    # meta: joined-on, used ID, location
    profile_meta = soup.find('dl', {'meta'})
    dts = [dt.text for dt in profile_meta.find_all('dt')]
    dds = [dd.text for dd in profile_meta.find_all('dd')]
    pairs = list(zip(dts, dds))
    for pair in pairs:
        location = ''
        if 'joined on' in pair[0]:
            joined_on = pair[1]
        elif 'user ID' in pair[0]:
            id_author = int(pair[1])
        elif 'live in' in pair[0]:
            location = pair[1]

    # create dictionary with author info
    author_info = {'author_link': author_link,
                   'author_name': author_name,
                   'id_author': id_author,
                   'joined_on': joined_on,
                   'location': location,
                   'works': works,
                   'series': series,
                   'gifts': gifts,
                   'bio': bio
                   }

    return id_author, author_info

In [7]:
def get_all_info(links_to_works):
    authors_dict = {}
    fanfics_dict = {}
    author_link_to_id = {}

    for link in tqdm(links_to_works):

        id_fanfic, fanfic_info = get_fanfic_info(link)
        author_link = fanfic_info['author_link']

        if author_link not in author_link_to_id:
            time.sleep(random.randint(2, 4))
            try:
                # https://archiveofourown.org/users/orphan_account/profile
                id_author, author_info = get_author_info(author_link)
                authors_dict[id_author] = author_info
            except AttributeError:
                id_author = None
                continue
            author_link_to_id[author_link] = id_author

        if author_link_to_id[author_link]:
            fanfic_info['id_author'] = author_link_to_id[author_link]
        else:
            fanfic_info['id_author'] = None

        fanfics_dict[id_fanfic] = fanfic_info

    time.sleep(random.randint(3, 5))
    return authors_dict, fanfics_dict

In [8]:
authors_dict, fanfics_dict = get_all_info(links_to_works)

  0%|          | 0/60 [00:00<?, ?it/s]

## Создание базы данных
![title](fanfics_schema.png)

### Подключение к базе

In [3]:
with open('mysql_password.txt', 'r', encoding='utf-8') as f:
    pwd = f.read().strip()

In [4]:
con = mysql.connector.connect(host='127.0.0.1', port=3306,
                              user='root', password=pwd)
cur = con.cursor(dictionary=True)

In [5]:
# cur.execute("""DROP SCHEMA IF EXISTS fanfics""")
# con.commit()

In [6]:
cur.execute("""
CREATE SCHEMA IF NOT EXISTS fanfics DEFAULT CHARACTER SET utf8
""")

In [7]:
con = mysql.connector.connect(host='127.0.0.1', port=3306,
                              database='fanfics', user='root',
                              password=pwd)
cur = con.cursor(dictionary=True, buffered=True)

### Фиксированные данные
#### Rating, category, warnings
У каждой работы есть рейтинг, категория и предупреждения. Эти данные представляют собой закрытые классы, которые легко задать вручную (как отдельные таблицы). Поскольку мы знаем, что новых данных в этих таблицах не появится, зададим длину поля как длину максимальной последовательности (это ускорит работу базы).

В каждой таблице три колонки:
* уникальный id (задается автоматически)
* название метки
* описание метки

In [14]:
ratings = [('Not Rated', 'This is the default option. For searching, screening, and other AO3 functions, this may get treated the same way as mature and explicit-rated content.'),
           ('General Audiences', 'The content is unlikely to be disturbing to anyone, and is suitable for all ages.'),
           ('Teen And Up Audiences', 'The content may be inappropriate for audiences under 13.'),
           ('Mature', "The content contains adult themes (sex, violence, etc.) that aren't as graphic as explicit-rated content."),
           ('Explicit', 'The content contains explicit adult themes, such as porn, graphic violence, etc.')]

In [15]:
def get_max_len(values):
    return max([len(val) for val in values])

In [16]:
rt_max_len = get_max_len([rt[0] for rt in ratings])
rt_desc_max_len = get_max_len([rt[1] for rt in ratings])

In [17]:
cur.execute("""
CREATE TABLE IF NOT EXISTS ratings (
  id_rating INT NOT NULL AUTO_INCREMENT,
  rating_title VARCHAR({}) NOT NULL,
  rating_desc VARCHAR({}) NULL,
  PRIMARY KEY (id_rating),
  UNIQUE INDEX id_rating_UNIQUE (id_rating ASC) VISIBLE,
  UNIQUE INDEX rating_title_UNIQUE (rating_title ASC) VISIBLE);
  """.format(rt_max_len, rt_desc_max_len))

In [18]:
for rating in ratings:
    cur.execute(
        """INSERT IGNORE INTO ratings (rating_title, rating_desc)
           VALUES ("{}", "{}")""".format(rating[0], rating[1]))
    con.commit()

In [19]:
categories = [('F/F', 'Female/Female relationships.'),
              ('F/M', 'Female/Male relationships.'),
              ('Gen', "General: no romantic or sexual relationships, or relationships which aren't the main focus of the work."),
              ('M/M', 'Male/Male relationships.'),
              ('Multi', 'More than one kind of relationship or a relationship with multiple partners.'),
              ('Other', 'Relationships not covered by the other categories.')]

In [20]:
ct_max_len = get_max_len([ct[0] for ct in categories])
ct_desc_max_len = get_max_len([ct[1] for ct in categories])

In [21]:
cur.execute("""
CREATE TABLE IF NOT EXISTS categories (
  id_category INT NOT NULL AUTO_INCREMENT,
  category_title VARCHAR({}) NOT NULL,
  category_desc VARCHAR({}) NULL,
  PRIMARY KEY (id_category),
  UNIQUE INDEX id_category_UNIQUE (id_category ASC) VISIBLE,
  UNIQUE INDEX category_title_UNIQUE (category_title ASC) VISIBLE);
  """.format(ct_max_len, ct_desc_max_len))

In [22]:
for category in categories:
    cur.execute(
        """INSERT IGNORE INTO categories (category_title, category_desc)
           VALUES ("{}", "{}")""".format(category[0], category[1]))
    con.commit()

In [23]:
warnings = [('Graphic Depictions Of Violence', 'The content contains gory, graphic, explicitly described violence.'),
            ('Major Character Death', "The content contains the death of a major character. Whether or not a character counts as a major character is up to the creator's discretion."),
            ('Rape/Non-Con', 'The content contains non-consensual sexual activity.'),
            ('Underage', 'The content contains graphic descriptions or depictions of sexual activity by characters under the age of eighteen.'),
            ('Creator Chose Not To Use Archive Warnings', "Warnings may apply but the creator doesn't want to use them (for example, to avoid spoilers)."),
            ('No Archive Warnings Apply', 'None of the warnings apply to the content.')]

In [24]:
wg_max_len = get_max_len([wg[0] for wg in warnings])
wg_desc_max_len = get_max_len([wg[1] for wg in warnings])

In [25]:
cur.execute("""
CREATE TABLE IF NOT EXISTS warnings_table (
  id_warning INT NOT NULL AUTO_INCREMENT,
  warning_title VARCHAR({}) NOT NULL,
  warning_desc VARCHAR({}) NULL,
  PRIMARY KEY (id_warning),
  UNIQUE INDEX id_warning_UNIQUE (id_warning ASC) VISIBLE,
  UNIQUE INDEX warning_title_UNIQUE (warning_title ASC) VISIBLE);
  """.format(wg_max_len, wg_desc_max_len))

In [26]:
for warning in warnings:
    cur.execute(
        """INSERT IGNORE INTO warnings_table (warning_title, warning_desc)
           VALUES ("{}", "{}")""".format(warning[0], warning[1]))
    con.commit()

### Нефиксированные данные
#### Authors
Для авторов нужно хранить следующую информацию:
* уникальный id (в системе AO3)
* username -- не больше 40 символов (*Due to the Archive's technical requirements, usernames \[...\] must begin and end with a letter or a number, and they must be at least 3 characters and **no more than 40 characters in length**.*)
* ссылка на профиль
* число работ, сборников, подарков
* дата регистрации
* место проживания
* описание профиля (bio) -- до 2000 символов (*Your About Me can contain **a maximum of 2000 characters** including limited HTML formatting.*)

In [27]:
cur.execute("""
CREATE TABLE IF NOT EXISTS authors (
  id_author INT NOT NULL,
  author_name VARCHAR(45) NOT NULL,
  author_link VARCHAR(255) NOT NULL,
  works INT NULL,
  series INT NULL,
  gifts INT NULL,
  joined_on DATE NULL,
  location VARCHAR(255) NULL,
  bio VARCHAR(2048) NULL,
  PRIMARY KEY (id_author),
  UNIQUE INDEX id_author_UNIQUE (id_author ASC) VISIBLE,
  UNIQUE INDEX author_link_UNIQUE (author_link ASC) VISIBLE,
  UNIQUE INDEX author_name_UNIQUE (author_name ASC) VISIBLE);
  """)

In [28]:
def format_for_mysql(some_value):
    if type(some_value) == str:
        return some_value.replace("'", "\\'").replace('"', "\\'")
    elif type(some_value) == list:
        new_value = [val.replace("'", "\\'").replace('"', "\\'")
                     for val in some_value]
        return new_value
    else:
        return some_value

In [29]:
for (id_author, author_info) in list(authors_dict.items()):
    author_info['bio'] = format_for_mysql(author_info['bio'])
    cur.execute(
        """INSERT IGNORE INTO authors (
        author_link, author_name, id_author, joined_on, location,
        works, series, gifts, bio)
        VALUES ("{}", "{}", "{}", "{}", NULLIF("{}", ''),
        "{}", "{}", "{}", NULLIF("{}", ''))
        """.format(*list(author_info.values())))
    con.commit()

#### Fanfics: texts & meta
Работа может состоять из одной или нескольких глав. Будем рассматривать каждую главу как отдельный текст, при этом каждый текст принадлежит какой-то одной работе.

Для работ нужно хранить следующую информацию:
* уникальный id (в системе AO3)
* ссылка на работу
* название
* автор (в виде уникального id)
* саммари (краткое описание всей работы), если есть
* рейтинг и категория в виде ссылки на соответствующую таблицу
* кол-во слов и глав
* статус (в процессе / завершен) в виде 0 или 1
* язык, на котором написана работа
* дата публикации, дата обновления
* кол-во лайков и просмотров

Отдельно хранятся фандомы, персонажи, пейринги, предупреждения и метки; работы связываются с ними путем таблиц соответствия.

Для глав (текстов) нужно хранить:
* уникальный id главы в нашей системе (auto-increment)
* уникальный id работы, частью которой является глава
* номер главы (0, если едиснственная, или от 1 до N)
* название главы (если не единственная)
* ссылка на главу
* саммари главы, если есть
* текст

Для начала создаем все нужные таблицы. Почти у всех текстовых полей с названиями установлен лимит 255 символов, у саммари -- 1250.

In [30]:
cur.execute("""
CREATE TABLE IF NOT EXISTS fandoms (
  id_fandom INT NOT NULL AUTO_INCREMENT,
  fandom_title VARCHAR(255) NOT NULL,
  PRIMARY KEY (id_fandom),
  UNIQUE INDEX id_fandom_UNIQUE (id_fandom ASC) VISIBLE,
  UNIQUE INDEX fandom_title_UNIQUE (fandom_title ASC) VISIBLE);
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS pairings (
  id_pairing INT NOT NULL AUTO_INCREMENT,
  pairing_title VARCHAR(255) NOT NULL,
  PRIMARY KEY (id_pairing),
  UNIQUE INDEX id_pairing_UNIQUE (id_pairing ASC) VISIBLE,
  UNIQUE INDEX pairing_title_UNIQUE (pairing_title ASC) VISIBLE);
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS characters (
  id_character INT NOT NULL AUTO_INCREMENT,
  character_name VARCHAR(255) NOT NULL,
  id_fandom INT NULL,
  PRIMARY KEY (id_character),
  UNIQUE INDEX id_character_UNIQUE (id_character ASC) VISIBLE,
  UNIQUE INDEX character_name_UNIQUE (character_name ASC) VISIBLE);
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS tags (
  id_tag INT NOT NULL AUTO_INCREMENT,
  tag_title VARCHAR(255) NOT NULL,
  PRIMARY KEY (id_tag),
  UNIQUE INDEX id_tag_UNIQUE (id_tag ASC) VISIBLE,
  UNIQUE INDEX tag_title_UNIQUE (tag_title ASC) VISIBLE);
  """)

con.commit()

In [31]:
cur.execute("""
CREATE TABLE IF NOT EXISTS works (
  id_fanfic INT NOT NULL,
  fanfic_link VARCHAR(255) NOT NULL,
  fanfic_title VARCHAR(255) NOT NULL,
  id_author INT NOT NULL,
  fanfic_summary VARCHAR(1250) NULL,
  rating INT NULL,
  category INT NULL,
  word_count INT NULL,
  chapter_count INT NULL,
  is_complete INT NULL,
  language VARCHAR(45) NULL,
  pub_date DATE NULL,
  upd_date DATE NULL,
  kudos INT NULL,
  hits INT NULL,
  PRIMARY KEY (id_fanfic),
  UNIQUE INDEX id_fanfic_UNIQUE (id_fanfic ASC) VISIBLE,
  UNIQUE INDEX fanfic_link_UNIQUE (fanfic_link ASC) VISIBLE,
  FOREIGN KEY (id_author) REFERENCES authors (id_author),
  FOREIGN KEY (rating) REFERENCES ratings (id_rating),
  FOREIGN KEY (category) REFERENCES categories (id_category));
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS chapters (
  id_chapter INT NOT NULL AUTO_INCREMENT,
  id_fanfic INT NOT NULL,
  chapter_num INT NOT NULL,
  chapter_name VARCHAR(255) NULL,
  chapter_link VARCHAR(255) NULL,
  chapter_summary VARCHAR(1250) NULL,
  chapter_text MEDIUMTEXT NULL,
  PRIMARY KEY (id_chapter),
  UNIQUE INDEX id_chapter_UNIQUE (id_chapter ASC) VISIBLE,
  FOREIGN KEY (id_fanfic) REFERENCES works (id_fanfic));
  """)

con.commit()

#### Таблицы связей
Нужно связать работы с:
* предупреждениями
* метками
* фандомами
* персонажами
* пейрингами

In [32]:
cur.execute("""
CREATE TABLE IF NOT EXISTS fanfic_to_warning (
  id_fanfic INT NOT NULL,
  id_warning INT NOT NULL,
  FOREIGN KEY (id_fanfic) REFERENCES works (id_fanfic),
  FOREIGN KEY (id_warning) REFERENCES warnings_table (id_warning));
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS fanfic_to_tag (
  id_fanfic INT NOT NULL,
  id_tag INT NOT NULL,
  FOREIGN KEY (id_fanfic) REFERENCES works (id_fanfic),
  FOREIGN KEY (id_tag) REFERENCES tags (id_tag));
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS fanfic_to_fandom (
  id_fanfic INT NOT NULL,
  id_fandom INT NOT NULL,
  FOREIGN KEY (id_fanfic) REFERENCES works (id_fanfic),
  FOREIGN KEY (id_fandom) REFERENCES fandoms (id_fandom));
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS fanfic_to_character (
  id_fanfic INT NOT NULL,
  id_character INT NOT NULL,
  FOREIGN KEY (id_fanfic) REFERENCES works (id_fanfic),
  FOREIGN KEY (id_character) REFERENCES characters (id_character));
  """)

cur.execute("""
CREATE TABLE IF NOT EXISTS fanfic_to_pairing (
  id_fanfic INT NOT NULL,
  id_pairing INT NOT NULL,
  FOREIGN KEY (id_fanfic) REFERENCES works (id_fanfic),
  FOREIGN KEY (id_pairing) REFERENCES pairings (id_pairing));
  """)

con.commit()

### Заполнение таблиц

In [33]:
for (id_fanfic, fanfic_info) in tqdm(list(fanfics_dict.items())):

    # приводим дату в правильный формат
    if not fanfic_info['upd_date']:
        fanfic_info['upd_date'] = '0000-00-00'

    # приводим рейтинг и категорию в числовой формат
    rating = fanfic_info['rating']
    if type(rating) == str:
        cur.execute("""
                    SELECT id_rating FROM ratings
                    WHERE rating_title='{}'""".format(fanfic_info['rating']))
        fanfic_info['rating'] = cur.fetchone()['id_rating']

    category = fanfic_info['category']
    if type(category) != int:
        if category:
            cur.execute("""
                        SELECT id_category FROM categories
                        WHERE category_title='{}'""".format(category))
            fanfic_info['category'] = cur.fetchone()['id_category']

    # выбираем нужные поля для записи в таблицу
    insert_into_works = ['id_fanfic', 'fanfic_link', 'title',
                         'summary', 'rating', 'language',
                         'pub_date', 'upd_date', 'words', 'chapt_num',
                         'status', 'kudos', 'hits', 'id_author']
    values_for_works = [format_for_mysql(fanfic_info[key])
                        for key in insert_into_works]

    # тут я сначала все отладила, потом поставила IGNORE
    # чтобы не падало, если в новом наборе что-то повторяется
    cur.execute(
        """INSERT IGNORE INTO works (id_fanfic, fanfic_link,
        fanfic_title, fanfic_summary, rating, language,
        pub_date, upd_date, word_count, chapter_count, is_complete,
        kudos, hits, id_author)
        VALUES ("{}", "{}", "{}", NULLIF("{}", ''), "{}", "{}",
        "{}", "{}", "{}", "{}", "{}", "{}", "{}", "{}")""".format(
            *values_for_works))
    if fanfic_info['category']:
        cur.execute(
            """UPDATE works SET category = {}
            WHERE id_fanfic='{}'""".format(fanfic_info['category'],
                                           fanfic_info['id_fanfic']))
    else:
        cur.execute(
            """UPDATE works SET category = NULL
            WHERE id_fanfic='{}'""".format(fanfic_info['id_fanfic']))
    con.commit()

    if fanfic_info['warnings']:
        cur.execute("""
            SELECT * FROM fanfic_to_warning WHERE id_fanfic='{}'
            """.format(id_fanfic))
        if not cur.fetchone():
            for warning in fanfic_info['warnings']:
                warning = format_for_mysql(warning)
                cur.execute("""
                    SELECT id_warning FROM warnings_table WHERE warning_title='{}'
                    """.format(warning))
                id_warning = cur.fetchone()['id_warning']
                cur.execute("""INSERT INTO fanfic_to_warning (
                    id_fanfic, id_warning) VALUES ("{}", "{}")
                    """.format(id_fanfic, id_warning))
                con.commit()

    if fanfic_info['tags']:
        cur.execute("""
            SELECT * FROM fanfic_to_tag WHERE id_fanfic='{}'
            """.format(id_fanfic))
        if not cur.fetchone():
            for tag in fanfic_info['tags']:
                tag = format_for_mysql(tag)
                cur.execute("""
                    SELECT id_tag FROM tags WHERE tag_title='{}'
                    """.format(tag))
                if not cur.fetchone():
                    cur.execute("""
                        INSERT INTO tags (tag_title)
                        VALUES ("{}")""".format(tag))
                cur.execute("""
                    SELECT id_tag FROM tags WHERE tag_title='{}'
                    """.format(tag))
                id_tag = cur.fetchone()['id_tag']
                cur.execute("""INSERT INTO fanfic_to_tag (
                    id_fanfic, id_tag) VALUES ("{}", "{}")
                    """.format(id_fanfic, id_tag))
                con.commit()

    if fanfic_info['fandoms']:
        cur.execute("""
            SELECT * FROM fanfic_to_fandom WHERE id_fanfic='{}'
            """.format(id_fanfic))
        if not cur.fetchone():
            for fandom in fanfic_info['fandoms']:
                fandom = format_for_mysql(fandom)
                cur.execute("""
                    SELECT id_fandom FROM fandoms WHERE fandom_title='{}'
                    """.format(fandom))
                if not cur.fetchone():
                    cur.execute("""
                        INSERT INTO fandoms (fandom_title)
                        VALUES ("{}")""".format(fandom))
                cur.execute("""
                    SELECT id_fandom FROM fandoms WHERE fandom_title='{}'
                    """.format(fandom))
                id_fandom = cur.fetchone()['id_fandom']
                cur.execute("""INSERT INTO fanfic_to_fandom (
                    id_fanfic, id_fandom) VALUES ("{}", "{}")
                    """.format(id_fanfic, id_fandom))
                con.commit()

    if fanfic_info['characters']:
        cur.execute("""
            SELECT * FROM fanfic_to_character WHERE id_fanfic='{}'
            """.format(id_fanfic))
        if not cur.fetchone():
            for character in fanfic_info['characters']:
                character = format_for_mysql(character)
                cur.execute("""
                    SELECT id_character FROM characters
                    WHERE character_name='{}'
                    """.format(character))
                if not cur.fetchone():
                    cur.execute("""
                        INSERT INTO characters (character_name)
                        VALUES ("{}")""".format(character))
                cur.execute("""
                    SELECT id_character FROM characters
                    WHERE character_name='{}'
                    """.format(character))
                id_character = cur.fetchone()['id_character']
                cur.execute("""INSERT INTO fanfic_to_character (
                    id_fanfic, id_character) VALUES ("{}", "{}")
                    """.format(id_fanfic, id_character))
                con.commit()

    if fanfic_info['pairings']:
        cur.execute("""
            SELECT * FROM fanfic_to_pairing WHERE id_fanfic='{}'
            """.format(id_fanfic))
        if not cur.fetchone():
            for pairing in fanfic_info['pairings']:
                pairing = format_for_mysql(pairing)
                cur.execute("""
                    SELECT id_pairing FROM pairings WHERE pairing_title='{}'
                    """.format(pairing))
                if not cur.fetchone():
                    cur.execute("""
                        INSERT INTO pairings (pairing_title)
                        VALUES ("{}")""".format(pairing))
                cur.execute("""
                    SELECT id_pairing FROM pairings WHERE pairing_title='{}'
                    """.format(pairing))
                id_pairing = cur.fetchone()['id_pairing']
                cur.execute("""INSERT INTO fanfic_to_pairing (
                    id_fanfic, id_pairing) VALUES ("{}", "{}")
                    """.format(id_fanfic, id_pairing))
                con.commit()

    for chapter in fanfic_info['chapters']:
        for (key, value) in list(chapter.items()):
            new_value = format_for_mysql(value)
            if new_value is None or new_value == '':
                new_value = ''
            chapter[key] = new_value
        values_for_chapters = list(chapter.values())
        cur.execute(
            """INSERT IGNORE INTO chapters (id_fanfic, chapter_num,
            chapter_name, chapter_link, chapter_summary, chapter_text)
            VALUES ("{}", "{}", NULLIF("{}", ''), NULLIF("{}", ''),
            NULLIF("{}", ''), "{}")""".format(
                *values_for_chapters))
        con.commit()

  0%|          | 0/58 [00:00<?, ?it/s]

## Запросы
#### Ввод, модификация, удаление, выборка
**Добавление** [автора](https://archiveofourown.org/users/TourmalineQueen/profile):

In [8]:
def add_author(id_author, author_name, author_link,
               works='', series='', gifts='',
               joined_on='', location='', bio=''):
    # для пустых полей оставляем NULL
    cur.execute(
        """INSERT INTO authors
        (id_author, author_name, author_link, works,
        series, gifts, joined_on, location, bio)
        VALUES ("{}", "{}", "{}", NULLIF("{}", ''),
        NULLIF("{}", ''), NULLIF("{}", ''), NULLIF("{}", ''),
        NULLIF("{}", ''), NULLIF("{}", ''))""".format(
            id_author, author_name, author_link, works,
            series, gifts, joined_on, location, bio))
    con.commit()

In [9]:
add_author(id_author=342844,
           author_name='TourmalineQueen',
           author_link='https://archiveofourown.org/users/TourmalineQueen/profile')

In [10]:
cur.execute(
    """SELECT * FROM authors
    WHERE id_author={}""".format(342844))
cur.fetchone()

{'id_author': 342844,
 'author_name': 'TourmalineQueen',
 'author_link': 'https://archiveofourown.org/users/TourmalineQueen/profile',
 'works': None,
 'series': None,
 'gifts': None,
 'joined_on': None,
 'location': None,
 'bio': None}

**Модификация**: допустим, автор обновил какую-то информацию о себе, и мы хотим внести ее в базу. Для этого можно сделать более общую функцию, которая сработает с разными таблицами.

In [11]:
def update_table(table, column, new_value,
                 idx_col, idx_value):
    cur.execute(
        f"""UPDATE {table} SET {column}='{new_value}'
        WHERE {idx_col}='{idx_value}'""")
    con.commit()

In [12]:
update_table('authors', 'location', 'Moscow',
             'id_author', 342844)

In [13]:
cur.execute(
    """SELECT * FROM authors
    WHERE id_author={}""".format(342844))
cur.fetchone()

{'id_author': 342844,
 'author_name': 'TourmalineQueen',
 'author_link': 'https://archiveofourown.org/users/TourmalineQueen/profile',
 'works': None,
 'series': None,
 'gifts': None,
 'joined_on': None,
 'location': 'Moscow',
 'bio': None}

**Удаление** (тоже общая функция):

In [14]:
def delete_row(table, idx_col, idx_value):
    cur.execute(
        f"""DELETE FROM {table} WHERE {idx_col}={idx_value}""")
    con.commit()

In [15]:
delete_row('authors', 'id_author', 342844)

In [16]:
cur.execute(
    """SELECT * FROM authors
    WHERE id_author={}""".format(342844))
cur.fetchone()

**Выбор**: выведем всех авторов, у которых больше 100 работ.

In [17]:
cur.execute(
    """SELECT author_name, works FROM authors
    WHERE works > 100""")
cur.fetchall()

[{'author_name': 'thehoyden', 'works': 116},
 {'author_name': 'Nokomis', 'works': 206},
 {'author_name': 'entanglednow', 'works': 751},
 {'author_name': 'lalazee', 'works': 146},
 {'author_name': 'dollsome', 'works': 327},
 {'author_name': 'PitViperOfDoom', 'works': 110}]

#### Запросы с JOIN
Посчитаем, сколько раз встретился каждый тег (точно так же можно посчитать статистику для фандомов, персонажей и т.д.).

In [18]:
cur.execute(
    """
    SELECT tag_title, count(id_fanfic) seen
    FROM fanfic_to_tag
    JOIN tags ON fanfic_to_tag.id_tag = tags.id_tag
    GROUP BY fanfic_to_tag.id_tag
    ORDER BY count(id_fanfic) DESC
    LIMIT 10
    """)
pd.DataFrame(cur.fetchall())

,tag_title,seen
0,Slow Burn,15
1,Hurt/Comfort,12
2,Angst,10
3,Humor,9
4,Romance,9
5,Fluff,8
6,Slow Build,8
7,Hogwarts Eighth Year,6
8,Alternate Universe,6
9,First Kiss,5


#### Последовательный JOIN
Получим 10 самых популярных фандомов. Популярность фандома будем определять средним числом кликов (= переходов по ссылке), которое получают работы, относящиеся к этому фандому. Примечание: здесь мы учитываем только ту информацию, которая реально есть в базе, сейчас ее мало.

In [19]:
cur.execute(
    """
    SELECT fandom_title, count(works.id_fanfic) works_num,
    round(avg(hits)) hits_per_work
    FROM works
    JOIN fanfic_to_fandom
    ON works.id_fanfic = fanfic_to_fandom.id_fanfic
    JOIN fandoms
    ON fanfic_to_fandom.id_fandom = fandoms.id_fandom
    GROUP BY fandom_title
    ORDER BY hits_per_work DESC
    LIMIT 10
    """)
pd.DataFrame(cur.fetchall())

,fandom_title,works_num,hits_per_work
0,Video Blogging RPF,2,1697780
1,Minecraft (Video Game),2,1697780
2,Supernatural,1,1310064
3,Haikyuu!!,2,1242015
4,博君一肖,1,1177168
5,僕のヒーローアカデミア | Boku no Hero Academia | My Hero ...,10,917658
6,Marvel (Movies),1,902698
7,Guardians of the Galaxy - All Media Types,1,902698
8,Marvel,1,902698
9,Marvel (Comics),1,902698


#### SELECT с 3+ таблицами в условии выборки
* Выбираем работы со статусом "завершено", рейтингом не выше Teen And Up Audiences (т.е. только General Audiences или Teen And Up Audiences), объемом больше 10 000 слов
* Выводим название работы и полный текст
* Для этого нужно сопоставить главы с работами по id_fanfic, работы с рейтингами по id_rating и еще задать условия для рейтинга, статуса и объема.

In [20]:
cur.execute(
    """SELECT works.id_fanfic, works.fanfic_title,
    GROUP_CONCAT(chapters.chapter_text SEPARATOR '\n') full_text
    FROM chapters, works, ratings
    WHERE chapters.id_fanfic = works.id_fanfic
    AND works.rating = ratings.id_rating
    AND (ratings.rating_title = 'General Audiences'
    OR ratings.rating_title = 'Teen And Up Audiences')
    AND works.is_complete = 1
    AND works.word_count > 10000
    GROUP BY works.id_fanfic
    ORDER BY fanfic_title
    """)
pd.DataFrame(cur.fetchall())

,id_fanfic,fanfic_title,full_text
0,8738770,2am Knows All Secrets,The first incident happened the Sunday that Cl...
1,11066478,Burn and Breathe,"For Shouto, it starts with burns marring his s..."
2,25069339,Cheat Code: Support Strategist,Izuku numbly watched the door shut behind All ...
3,22080895,Deku? I think he's some pro...,"Even after an hour-long shower, Izuku still fe..."
4,20049589,Evitative,In the summer before his fifth year at Hogwart...
5,7331278,Hermione Granger's Hogwarts Crammer for Delinq...,Draco Malfoy’s shiny dragon-leather oxford sun...
6,14446512,Hero Class Civil Warfare,1 hour until the exam begins\nThe sounds of me...
7,3195734,Home,Stiles stares down at the ivory bistro mug he ...
8,10920276,If You Had This Time Again,Tony had always considered the saying hearing ...
9,5096105,In Another Life,It had been some time since Akaashi set foot i...


Занимает 0.015 сек в MySQL Workbench

#### То же, что выше, через JOIN:

In [21]:
cur.execute(
    """
    SELECT chapters.id_fanfic, fanfic_title, GROUP_CONCAT(chapter_text SEPARATOR '\n') full_text
    FROM chapters
    JOIN (works JOIN ratings ON works.rating = ratings.id_rating)
    ON chapters.id_fanfic = works.id_fanfic
    WHERE (rating_title = "General Audiences"
    OR rating_title = "Teen And Up Audiences")
    AND is_complete = 1
    AND word_count > 10000
    GROUP BY chapters.id_fanfic
    ORDER BY fanfic_title
    """)
pd.DataFrame(cur.fetchall())

,id_fanfic,fanfic_title,full_text
0,8738770,2am Knows All Secrets,The first incident happened the Sunday that Cl...
1,11066478,Burn and Breathe,"For Shouto, it starts with burns marring his s..."
2,25069339,Cheat Code: Support Strategist,Izuku numbly watched the door shut behind All ...
3,22080895,Deku? I think he's some pro...,"Even after an hour-long shower, Izuku still fe..."
4,20049589,Evitative,In the summer before his fifth year at Hogwart...
5,7331278,Hermione Granger's Hogwarts Crammer for Delinq...,Draco Malfoy’s shiny dragon-leather oxford sun...
6,14446512,Hero Class Civil Warfare,1 hour until the exam begins\nThe sounds of me...
7,3195734,Home,Stiles stares down at the ivory bistro mug he ...
8,10920276,If You Had This Time Again,Tony had always considered the saying hearing ...
9,5096105,In Another Life,It had been some time since Akaashi set foot i...


Занимает 0.016 сек в MySQL Workbench

#### Агрегация, сортировки + вложенный SELECT
Выясним, сколько тегов в среднем имеет каждая работа.

In [22]:
cur.execute(
    """SELECT round(avg(tag_num)) as avg_tag_num
    FROM (
    SELECT works.fanfic_title, count(*) as tag_num
    FROM works, fanfic_to_tag
    WHERE works.id_fanfic = fanfic_to_tag.id_fanfic
    GROUP BY works.id_fanfic) as t1
    """)
cur.fetchone()

{'avg_tag_num': Decimal('15')}

Посчитаем, сколько лайков в среднем получает работа каждого автора. Будем сортировать выдачу по среднему числу лайков (так мы определяем популярность автора). Примечание: при подсчете мы опираемся только на те данные, которые есть в базе.

In [23]:
cur.execute(
    """SELECT authors.author_name, count(works.id_fanfic) known_works,
    round(avg(works.kudos)) kudos_per_work
    FROM works, authors
    WHERE works.id_author = authors.id_author
    GROUP BY works.id_author
    ORDER BY avg(works.kudos) DESC
    LIMIT 10
    """)
pd.DataFrame(cur.fetchall())

,author_name,known_works,kudos_per_work
0,sherlocksmyth,1,108743
1,MsKingBean89,1,78364
2,blujamas,1,70040
3,LittleLuxray,1,63948
4,treezie,1,49869
5,entanglednow,1,43531
6,Icarius51,1,43187
7,PitViperOfDoom,4,42492
8,Sharleena,1,41480
9,MuffinLance,1,39257


#### Вложенный SELECT, WITH, агрегация

In [24]:
# здесь убрать последний WITH и включить в него сортировку
cur.execute(
    """WITH category_list AS
     (SELECT authors.author_name, categories.category_title,
     COUNT(*) AS category_count
        FROM authors
        JOIN works
          ON authors.id_author = works.id_author
        JOIN categories
          ON works.category = categories.id_category
       GROUP BY authors.id_author, categories.category_title
     ),
     auth_to_cat AS
     (SELECT category_list.author_name, category_list.category_title
       FROM category_list
       JOIN (SELECT category_list.author_name, MAX(category_list.category_count) AS max_cat_count
               FROM category_list
               GROUP BY category_list.author_name
      ) AS t6
    ON category_list.author_name = t6.author_name AND category_list.category_count = t6.max_cat_count)
    
    SELECT * FROM auth_to_cat
    ORDER BY author_name
    """)
pd.DataFrame(cur.fetchall())

,author_name,category_title
0,Annacharlier,F/F
1,arysteia,M/M
2,bafflinghaze,M/M
3,blujamas,Gen
4,CheckeredCloth,M/M
5,cyerus,M/M
6,Deshonanana,M/M
7,dls,M/M
8,dollsome,M/M
9,drannie,M/M
